In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab09.ipynb")

## Lab 09: Classification

Welcome to Foundations of Data Science for High School! Throughout the course you will complete assignments like this one. You can't learn technical subjects without hands-on practice, so these assignments are an important part of the course.

**Collaboration Policy:**

Collaborating on labs is more than okay -- it's encouraged! You should rarely remain stuck for more than a few minutes on questions in labs, so ask a neighbor or an instructor for help. Explaining things is beneficial, too -- the best way to solidify your knowledge of a subject is to explain it. You should **not** _just_ copy/paste someone else's code, but rather work together to gain understanding of the task you need to complete. 

**Due Date:** 

## Today's Assignment

In today's lab, we will lget some hands-on practice with another predictive modeling problem $-$ classification. Specifically, we will look at using k-Nearest Neighbors classification (kNN).

For all problems that you must write our explanations and sentences for, you **must** provide your answer in the designated space. Also, throughout this lab please be sure to not re-assign variables throughout the notebook. For example, if you use `expected_proportion_correct` in your answer to one question, do not reassign it later on. Moreover, be mindful that functions that you write in earlier cells will need to be referenced to answer questions in later questions.

Recommended reading:

- [Classification](https://inferentialthinking.com/chapters/17/Classification.html)

In [ ]:
import numpy as np
from datascience import *

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

## 1. Tobacco Road Coordinates with Classification


### Our Dearest Neighbors

Carol is trying classify students as either attendees of Duke University or UNC at Chapel Hill. To classify the students, Carol has access to the coordinates of the location they live during the school year (wow, kind of creepy Carol). First, load in the `coordinates` table.

Run the cell below.

In [ ]:
coordinates = Table.read_table('data/coordinates_nc.csv')
coordinates.show(5)

As usual, let's investigate our data visually before performing any kind of numerical analysis.

Run the cell below.

In [ ]:
coordinates.scatter("longitude", "latitude", group="school")

In this case, most people probably don't recognize how the latitude and longitude relate to real life, so we can use a mapping function to put these points onto a map.

Run the cell below.

In [ ]:
colors = {"Duke":"darkblue", "UNC-CH":"cornflowerblue"}
t = Table().with_columns("lat", coordinates.column(0), 
                                      "lon", coordinates.column(1), 
                                      "color", coordinates.apply(colors.get, 2)
                        )
Circle.map_table(t, area=5, fill_opacity=1)

**Question 1.**  Let's begin implementing the k-Nearest Neighbors algorithm. Define the `distance` function, which takes in two arguments: an array of numerical features, and a different array of numerical features. The function should return the [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between the two arrays. Euclidean distance is often referred to as the straight-line distance formula that you may have learned previously.

In [ ]:
def distance(arr1, arr2):
    """Return the Euclidean distance between the two arrays."""
    ...

# Don't change/delete the code below.
distance_example = distance(make_array(1, 2, 3), make_array(4, 5, 6))
distance_example

In [ ]:
grader.check("q1")

### Splitting the dataset

We'll do 2 different kinds of things with the `coordinates` dataset:

1. We'll build a classifier using coordinates for which we know the associated label; this will teach it to recognize labels of similar coordinate values. This process is known as *training*.

2. We'll evaluate or *test* the accuracy of the classifier we build on data we haven't seen before.

For reasons discussed in class and the textbook, we want to use separate datasets for these two purposes.  So we split up our one dataset into two.

**Question 2.** Next, let's split our dataset into a training set and a test set. Since `coordinates` has 84 rows, let's create a training set with the first 63 rows (75% of the data) and a test set with the remaining 21 rows (25% of the data). Remember that assignment to each group should be random, so you should shuffle the table **first** (save it to `shuffled_table`), then select 63 rows for the training set and 21 rows for the testing set using the `.take()` table method with `np.arange` to specify which rows go into each set.

In [ ]:
shuffled_table = ...
train = ...
test = ...

# Don't change/delete the code below.
print("Training set:\t",   train.num_rows, "examples")
train.show(5)

print("Test set:\t",       test.num_rows, "examples")
test.show(5);

In [ ]:
grader.check("q2")

**Question 3.** Assign `features` to an array of the labels of the features from the `coordinates` table.

**Hint:** Which of the column labels in the `coordinates` table are the features, and which of the column labels correspond to the class we're trying to predict?

In [ ]:
features = ...
features

In [ ]:
grader.check("q3")

**Question 4.** Now define the `classify` function. This function should take in a `row` from a table like `test` and classify it based on the data in `train` using the `k`-Nearest Neighbors based on the correct `features`.

**Hints:** 

- Use the `row_to_array` function we defined for you to convert rows to arrays of features so that you can use the `distance` function you defined earlier.

- The skeleton code we provided iterates through each row in the training set.

In [ ]:
def row_to_array(row, features):
    """Convert rows to arrays of features."""
    arr = make_array()
    for feature in features:
        arr = np.append(arr, row.item(feature))
    return arr

def classify(row, k, train):
    """Iterate through each row in the training set and return a prediction (classify)
        based in the correct features."""
    test_row_features_array = row_to_array(row, features)
    distances = make_array()
    for train_row in train.rows:
        train_row_features_array = row_to_array(train_row, features) # SOLUTION
        row_distance = distance(test_row_features_array, train_row_features_array) # SOLUTION
        distances = np.append(distances, row_distance) # SOLUTION
    train_with_distances = train.with_column('Distances', distances) # SOLUTION
    nearest_neighbors = train_with_distances.sort('Distances').take(np.arange(k))
    most_common_label = nearest_neighbors.group('school').sort('count', descending=True).column('school').item(0) # SOLUTION
    return most_common_label # SOLUTION

# The code below will attempt to classify the first row in your 
# test dataset using a 5 nearest neighbors classifier
first_test = classify(test.row(0), 5, train)
first_test

In [ ]:
grader.check("q4")

**Question 5.** The function `three_classify` takes in a `row` from `test` as an argument and classifies the row using a 3-Nearest Neighbors classifier. We define this function so we can use the `apply()` method to quickly classify all the rows we have in the testing data set. We can then compare the prediction from the classifier to the known label for each row to get an idea of how accurate our classifier is on the test data. 

The `test_with_prediction` table is created for you. Use this table to find the `accuracy` of a 3-NN classifier on the `test` set. `accuracy` should be a proportion (not a percentage) of the schools that were correctly predicted.

In [ ]:
def three_classify(row):
    """Take a row from test and classifies the row using a 
       3-Nearest Neighbors classifier."""
    return classify(row, 3, train)

test_with_prediction = test.with_column("prediction", test.apply(three_classify))
labels_correct = ...
accuracy = ...
accuracy

In [ ]:
grader.check("q5")

<!-- BEGIN QUESTION -->

**Question 6.** Suppose you work at the leasing office for an apartment building located at the GPS coordinates 35.95979476700251, -78.9870877612551 and a student is moving in. The cell below will create a table `new_student` with these coordinates as the only row.

What school would your classifier predict they attend using a 3 nearest neighbor model?

In [ ]:
new_student = Table().with_columns('latitude', make_array(35.95979476700251), 'longitude', make_array(-78.9870877612551))
new_student

In [ ]:
...

<!-- END QUESTION -->

**Question 7.** There are 45 rows of Duke students and 39 rows of UNC-CH students in the `coordinates` table. If we used the entire `coordinates` table as the train set, what is the smallest value of k that would ensure that a k-Nearest Neighbor classifier would always predict Duke as the school? Assign the value to `k`. The test on this question will only verify your answer is formatted correctly, it will not check it for accuracy. 

In [ ]:
k = ...
k

In [ ]:
grader.check("q7")

<!-- BEGIN QUESTION -->

**Question 8.** Why do we divide our data into a training and test set instead of using all the data to train the model?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 9.** Why do we use an odd-numbered `k` in k-NN? Explain.

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)